## ETL Project by Moe, Fernando and Laura
Goals: 1.) To identify which neighborhoods in NYC have high population density, both from residents and tourists, yet do not have enough food establishments nearby. 2.) To provide new restaurants with guidance on strategic placements in NYC.

### Import All Dependencies

In [1]:
import gmaps
# import numpy as np
import pandas as pd
import requests
# import time
import json
# from datetime import datetime
# from scipy.stats import linregress
# from matplotlib import pyplot as plt
import os
from bs4 import BeautifulSoup as bs

# Import API key
from config import g_key

### This section will be used to clean the datasets
Datasets used: 3 CSV Files (one containing Airbnb data from Kaggle, one on census data for NYC from https://data.cccnewyork.org/ and one for restaurant data for NYC from Kaggle). In addition, we will webscrape an article on the top 25 restaurants to stay in NYC.

#### A. Cleaning census data for NYC

In [2]:
# import csv for census data and remove the first 5 rows that contain other information
census_data = pd.read_csv('Resources/NYC_census_data.csv', skiprows=5)

In [3]:
census_data.head(5)

,Location,TimeFrame,DataFormat,Data,Fips
0,Battery Park/Tribeca,2005,Number,51150.45963,101
1,Battery Park/Tribeca,2006,Number,59445.00870,101
2,Battery Park/Tribeca,2007,Number,56375.39151,101
3,Battery Park/Tribeca,2008,Number,61639.90009,101
4,Battery Park/Tribeca,2009,Number,56591.85182,101


In [4]:
#remove dataformat and fips from the dataset
census_data = census_data.drop(['DataFormat', 'Fips'], axis=1)
census_data.head()

,Location,TimeFrame,Data
0,Battery Park/Tribeca,2005,51150.45963
1,Battery Park/Tribeca,2006,59445.00870
2,Battery Park/Tribeca,2007,56375.39151
3,Battery Park/Tribeca,2008,61639.90009
4,Battery Park/Tribeca,2009,56591.85182


In [5]:
#remove data that started with "Zip Code" as it does not represent a neighborhood
census_data = census_data[~census_data.Location.str.startswith('Zip')]

In [6]:
#rename column headers
census_data = census_data.rename(columns = {'Location': 'Neighborhood', 'TimeFrame': 'Census_Year', 'Data': 'Population'}, inplace = False)
census_data.head()

,Neighborhood,Census_Year,Population
0,Battery Park/Tribeca,2005,51150.45963
1,Battery Park/Tribeca,2006,59445.00870
2,Battery Park/Tribeca,2007,56375.39151
3,Battery Park/Tribeca,2008,61639.90009
4,Battery Park/Tribeca,2009,56591.85182


In [7]:
#sort by timeframe location and timeframe, and for each unique location select the latest year of census
census_data = census_data.sort_values(by = ['Neighborhood','Census_Year'], ascending = (True, False))
census_data = census_data.drop_duplicates(subset=['Neighborhood'])
census_data.head()

,Neighborhood,Census_Year,Population
2802,Astoria,2019,166069.00000
2760,Battery Park/Tribeca,2019,66437.96901
2793,Bay Ridge,2019,121925.00000
2812,Bayside,2019,114562.00000
1277,Bayside - Little Neck,2018,87431.63278


#### B. Getting from website the top 25 hotels in NYC

In [8]:
# get url
url = 'https://www.businessinsider.com/best-hotels-new-york-city'
# Retrieve page with the requests module
response = requests.get(url)

In [9]:
#  BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

In [10]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <title>
   Best Hotels in New York City (NYC) (Updated April 2021)
  </title>
  <script>
   function createPerformanceMark(e){void 0!==window.performance&&void 0!==window.performance.mark&&performance.mark(e)}function createPerformanceMeasure(e,r,o){void 0!==window.performance&&void 0!==window.performance.measure&&performance.measure(e,r,o)}
  </script>
  <!-- Speedcurve, LUX - Start -->
  <script>
   LUX=function(){var e="undefined"!=typeof LUX&&void 0!==LUX.gaMarks?LUX.gaMarks:[],n="undefined"!=typeof LUX&&void 0!==LUX.gaMeasures?LUX.gaMeasures:[],t="LUX_start",r=window.performance,a="undefined"!=typeof LUX&&LUX.ns?LUX.ns:Date.now?Date.now():+new Date;function u(){if(r){if(r.now)return r.now();if(r.webkitNow)return r.now();if(r.msNow)return r.now();if(r.mozNow)return r.now()}return(Date.now?Date.now():+new Date)-a}function o(n){return function(e,n){for(i=n.length-1;i>=0;i--){var t=n[i];if(e===t.name)return t}return}(n,function(){if(r){if(

In [11]:
# results are returned as an iterable list
results = soup.find_all('h2', class_='slide-title-text')

In [12]:
#create empty list to append restaurants
top_hotels = []

In [13]:
# Loop through returned results
for result in results:
    top_hotels.append(result.text)


In [14]:
#put list into dataframe and rename column header, select 25 top restaurants from article
top_hotels_df = pd.DataFrame(top_hotels)
top_hotels_df = top_hotels_df.head(25)
top_hotels_df.columns = ['Top Hotels']

In [52]:
#hotels to city
hotels_list = top_hotels_df['Top Hotels'].to_list()

In [84]:
# test = 'Arlo SoHo'
# base_url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={test}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key={g_key}"
# response = requests.get(base_url).json()
# response["candidates"][0]["formatted_address"][-20:-15]


'10013'

In [86]:
# set up lists to hold reponse info
hotels = []
lat = []
long = []
zip_code = []


# Loop through the list of hotels and perform a request for data on each
for hotel in hotels_list: 
    if hotel not in hotels:
        base_url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={hotel}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key={g_key}"
        response = requests.get(base_url).json()
        try:
                hotels.append(hotel)
                lat.append(response["candidates"][0]["geometry"]["location"]['lat'])
                long.append(response["candidates"][0]["geometry"]["location"]['lat'])
                zip_code.append(response["candidates"][0]["formatted_address"][-20:-15])
                           
        except:
            continue
top_hotels = {'Hotel Name':hotels,'Lat':lat,'Lon':long,'Zip Code':zip_code}
top_hotels_df = pd.DataFrame.from_dict(top_hotels, orient='index')
top_hotels_df = top_hotels_df.transpose()
top_hotels_df = top_hotels_df.dropna()
top_hotels_df.head()

,Hotel Name,Lat,Lon,Zip Code
0,Arlo SoHo,40.7245,40.7245,10013
1,"Gild Hall, a Thompson Hotel",40.7079,40.7079,10038
2,The Freehand New York,40.7398,40.7398,10010
3,Hotel 50 Bowery,40.7159,40.7159,10013
4,The Evelyn New York,40.7439,40.7439,10016
